In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm_notebook
from time import time

In [238]:
driver = webdriver.Chrome()

In [292]:
res = []
errors = []
for p in tqdm_notebook(range(1, 500)):
    try:
        url = 'https://www.glowpick.com/beauty/ranking?id={}'.format(p)

        driver.get(url)
        cate1 = driver.find_element_by_css_selector('#gp-list > section > h1 > span:nth-child(2)')
        cate1.click()

        body = driver.find_element_by_css_selector('body')
        
        st = time()
        while(True):
            ul = driver.find_element_by_css_selector('#gp-list > div > section.section-list > ul')
            body.send_keys(Keys.END)
            sleep(0.2)
            lis= ul.find_elements_by_css_selector('li .list-item') 
            if len(lis) >= 100:
                break 
            if time() - st > 7:
                break
            
        soup = BeautifulSoup(ul.get_attribute('innerHTML'), 'lxml')

        lis = soup.select('li .list-item') 

        for idx, li in enumerate(lis): 
            rank_info = li.select('div.side-ranking span')
            
            rank=change=diff ='' 
            if len(rank_info)> 1:
                rank = rank_info[0].text
                if len(rank_info) > 2: 
                    change = '/'.join(rank_info[1]['class'][2])
                    diff = rank_info[2].text
            
            info = li.select_one('.side-info')

            link = li.select_one('div.comp-card-prod')['data-url']
            link = link.replace('/product/','')

            brand = info.select_one('.side-prod .brand').text
            name = info.select_one('.side-prod .name').text
            volume = info.select_one('.prod-info .volume').text
            price = info.select_one('.prod-info .price').text

            res.append([cate1.text, p, rank, change, diff, link, brand, name, volume, price])
    except Exception as e:
        #print(e)
        #print(idx)
        errors.append(p)

In [293]:
df = pd.DataFrame(res, columns=['cate','cid','rank','change','diff','pn','company','name','volume','price'])

In [296]:
df.to_csv('data/rank.csv', index=False)

In [2]:
df = pd.read_csv('data/rank.csv')

In [5]:
df = df[df['rank'].notna()]

In [7]:
df.to_csv('data/rank.csv', index=False)